In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    #'post_padding', 'pre_padding', 
    #'mid_padding', 
    #             'strf_padding', 'ext_padding', 
    #             'rnd_padding', 
    'aug_padding', 
    "zoom_padding"]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["zoom_padding"]
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/' #'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#'EC_number/archaea/rnn_conv/256rnn/'
#'EC_number/archaea/1conv/64filts_size5/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task1/"
architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
#architecture = "stack_conv"

In [3]:
lr= 0.0001 #0.001#
optimizer = Adam(lr=lr) #Adam(lr=lr)

### Loading stuff

In [4]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_split.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_split.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model  - Task 1

In [5]:
model = building_3dense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
#model = building_stackconv_model_task1(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)
generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task1', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 314)               8164314   
_________________________________________________________________
dropout_2 (Dropout)          (None, 314)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 77)                24255     
_________________________________________________________________
dropout_3 (Dropout)          (None, 77)                0         
__________

### Training model

In [6]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, val_generator,
                        architecture, max_lenn, dict_size, batch_size,
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val, 
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 26000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 314)               8164314   
_________________________________________________________________
dropout_6 (Dropout)          (None, 314)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 77)                24255     
_________________________________________________________________
dropout_7 (Dropout)          (None, 77)                0         
__________

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1758/1758 [==============================] - 28s 16ms/step - loss: 0.2675 - acc: 0.9017 - val_loss: 0.3411 - val_acc: 0.8873

Epoch 00023: val_acc did not improve from 0.89460
Epoch 24/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.2646 - acc: 0.9022 - val_loss: 0.3357 - val_acc: 0.8881

Epoch 00024: val_acc did not improve from 0.89460
Epoch 25/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.2611 - acc: 0.9032 - val_loss: 0.3045 - val_acc: 0.9035

Epoch 00025: val_acc improved from 0.89460 to 0.90347, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/aug_padding/0/weights-improvement-025-0.9035.hdf5
Epoch 26/200
1758/1758 [==============================] - 28s 16ms/step - loss: 0.2532 - acc: 0.9075 - val_loss: 0.3240 - val_acc: 0.8911

Epoch 00026: val_acc did not improve from 0.90347
Epoch 27/200
1758/1758 [==============================] - 28s 16ms/step - loss: 0.2498 - acc: 0.9079 


Epoch 00056: val_acc did not improve from 0.95602
Epoch 57/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1713 - acc: 0.9385 - val_loss: 0.1838 - val_acc: 0.9345

Epoch 00057: val_acc did not improve from 0.95602
Epoch 58/200
1758/1758 [==============================] - 28s 16ms/step - loss: 0.1727 - acc: 0.9385 - val_loss: 0.1759 - val_acc: 0.9488

Epoch 00058: val_acc did not improve from 0.95602
Epoch 59/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1683 - acc: 0.9399 - val_loss: 0.1728 - val_acc: 0.9514

Epoch 00059: val_acc did not improve from 0.95602
Epoch 60/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1647 - acc: 0.9412 - val_loss: 0.1713 - val_acc: 0.9542

Epoch 00060: val_acc did not improve from 0.95602
Epoch 61/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1643 - acc: 0.9423 - val_loss: 0.1482 - val_acc: 0.9551

Epoch 00061: val_acc did not improve from 0.95602
Epoc

1758/1758 [==============================] - 27s 15ms/step - loss: 0.1426 - acc: 0.9509 - val_loss: 0.1344 - val_acc: 0.9579

Epoch 00093: val_acc did not improve from 0.96676
Epoch 94/200
1758/1758 [==============================] - 28s 16ms/step - loss: 0.1432 - acc: 0.9504 - val_loss: 0.1577 - val_acc: 0.9622

Epoch 00094: val_acc did not improve from 0.96676
Epoch 95/200
1758/1758 [==============================] - 28s 16ms/step - loss: 0.1437 - acc: 0.9506 - val_loss: 0.1478 - val_acc: 0.9631

Epoch 00095: val_acc did not improve from 0.96676
Epoch 96/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1443 - acc: 0.9508 - val_loss: 0.1406 - val_acc: 0.9661

Epoch 00096: val_acc did not improve from 0.96676
Epoch 97/200
1758/1758 [==============================] - 27s 15ms/step - loss: 0.1429 - acc: 0.9510 - val_loss: 0.1431 - val_acc: 0.9705

Epoch 00097: val_acc improved from 0.96676 to 0.97055, saving model to /home/angela/padding_EBI/data/checkpoint/EC_num

...

251/251 [==============================] - 4s 16ms/step - loss: 0.1259 - acc: 0.9233 - val_loss: 0.0148 - val_acc: 0.9972

Epoch 00018: val_acc improved from 0.99092 to 0.99720, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/zoom_padding/9/weights-improvement-018-0.9972.hdf5
Epoch 19/200
251/251 [==============================] - 4s 16ms/step - loss: 0.1124 - acc: 0.9289 - val_loss: 0.0098 - val_acc: 0.9979

Epoch 00019: val_acc improved from 0.99720 to 0.99790, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/zoom_padding/9/weights-improvement-019-0.9979.hdf5
Epoch 20/200
251/251 [==============================] - 4s 16ms/step - loss: 0.1098 - acc: 0.9243 - val_loss: 0.0060 - val_acc: 0.9990

Epoch 00020: val_acc improved from 0.99790 to 0.99895, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3denses/bio_neurons/task1/zoom_padding/9/weights-improvement-

251/251 [==============================] - 4s 16ms/step - loss: 0.0594 - acc: 0.9787 - val_loss: 2.9433e-05 - val_acc: 1.0000

Epoch 00057: val_acc did not improve from 1.00000
Epoch 58/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0514 - acc: 0.9818 - val_loss: 2.5314e-05 - val_acc: 1.0000

Epoch 00058: val_acc did not improve from 1.00000
Epoch 59/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0533 - acc: 0.9799 - val_loss: 1.7213e-05 - val_acc: 1.0000

Epoch 00059: val_acc did not improve from 1.00000
Epoch 60/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0494 - acc: 0.9832 - val_loss: 2.0837e-05 - val_acc: 1.0000

Epoch 00060: val_acc did not improve from 1.00000
Epoch 61/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0547 - acc: 0.9809 - val_loss: 2.0515e-05 - val_acc: 1.0000

Epoch 00061: val_acc did not improve from 1.00000
Epoch 62/200
251/251 [==============================] - 4s 16ms

251/251 [==============================] - 4s 16ms/step - loss: 0.0503 - acc: 0.9817 - val_loss: 1.3440e-06 - val_acc: 1.0000

Epoch 00100: val_acc did not improve from 1.00000
Epoch 101/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0520 - acc: 0.9811 - val_loss: 1.2938e-06 - val_acc: 1.0000

Epoch 00101: val_acc did not improve from 1.00000
Epoch 102/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0502 - acc: 0.9794 - val_loss: 6.9327e-07 - val_acc: 1.0000

Epoch 00102: val_acc did not improve from 1.00000
Epoch 103/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0459 - acc: 0.9827 - val_loss: 5.5468e-07 - val_acc: 1.0000

Epoch 00103: val_acc did not improve from 1.00000
Epoch 104/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0477 - acc: 0.9802 - val_loss: 6.1044e-07 - val_acc: 1.0000

Epoch 00104: val_acc did not improve from 1.00000
Epoch 105/200
251/251 [==============================] - 4s

251/251 [==============================] - 4s 16ms/step - loss: 0.0467 - acc: 0.9815 - val_loss: 1.8608e-07 - val_acc: 1.0000

Epoch 00143: val_acc did not improve from 1.00000
Epoch 144/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0471 - acc: 0.9819 - val_loss: 3.6605e-07 - val_acc: 1.0000

Epoch 00144: val_acc did not improve from 1.00000
Epoch 145/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0458 - acc: 0.9818 - val_loss: 3.0939e-07 - val_acc: 1.0000

Epoch 00145: val_acc did not improve from 1.00000
Epoch 146/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0464 - acc: 0.9811 - val_loss: 3.9426e-07 - val_acc: 1.0000

Epoch 00146: val_acc did not improve from 1.00000
Epoch 147/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0456 - acc: 0.9819 - val_loss: 3.6144e-07 - val_acc: 1.0000

Epoch 00147: val_acc did not improve from 1.00000
Epoch 148/200
251/251 [==============================] - 4s

251/251 [==============================] - 4s 16ms/step - loss: 0.0427 - acc: 0.9832 - val_loss: 1.7886e-07 - val_acc: 1.0000

Epoch 00186: val_acc did not improve from 1.00000
Epoch 187/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0474 - acc: 0.9811 - val_loss: 1.8350e-07 - val_acc: 1.0000

Epoch 00187: val_acc did not improve from 1.00000
Epoch 188/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0463 - acc: 0.9815 - val_loss: 1.6415e-07 - val_acc: 1.0000

Epoch 00188: val_acc did not improve from 1.00000
Epoch 189/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0416 - acc: 0.9838 - val_loss: 1.6357e-07 - val_acc: 1.0000

Epoch 00189: val_acc did not improve from 1.00000
Epoch 190/200
251/251 [==============================] - 4s 16ms/step - loss: 0.0469 - acc: 0.9816 - val_loss: 2.8017e-07 - val_acc: 1.0000

Epoch 00190: val_acc did not improve from 1.00000
Epoch 191/200
251/251 [==============================] - 4s